In [1]:
%matplotlib inline

In [67]:
import os
import pickle
from glob import glob
import pandas as pd
import numpy as np
import nibabel as nb
from nighres.io import io_mesh
import matplotlib.pyplot as plt
from nipype.interfaces import fsl
from scipy.spatial.distance import pdist
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
import h5py
from scipy.stats import stats

In [3]:
mcc = MouseConnectivityCache(manifest_file=
                             '/home/julia/data/gradients/atlas/allen_api/mouse_connectivity_manifest.json')

In [4]:
structure_tree = mcc.get_structure_tree()
structure_tree.get_name_map()

{997: 'root',
 8: 'Basic cell groups and regions',
 567: 'Cerebrum',
 688: 'Cerebral cortex',
 695: 'Cortical plate',
 315: 'Isocortex',
 184: 'Frontal pole, cerebral cortex',
 68: 'Frontal pole, layer 1',
 667: 'Frontal pole, layer 2/3',
 526157192: 'Frontal pole, layer 5',
 526157196: 'Frontal pole, layer 6a',
 526322264: 'Frontal pole, layer 6b',
 500: 'Somatomotor areas',
 107: 'Somatomotor areas, Layer 1',
 219: 'Somatomotor areas, Layer 2/3',
 299: 'Somatomotor areas, Layer 5',
 644: 'Somatomotor areas, Layer 6a',
 947: 'Somatomotor areas, Layer 6b',
 985: 'Primary motor area',
 320: 'Primary motor area, Layer 1',
 943: 'Primary motor area, Layer 2/3',
 648: 'Primary motor area, Layer 5',
 844: 'Primary motor area, Layer 6a',
 882: 'Primary motor area, Layer 6b',
 993: 'Secondary motor area',
 656: 'Secondary motor area, layer 1',
 962: 'Secondary motor area, layer 2/3',
 767: 'Secondary motor area, layer 5',
 1021: 'Secondary motor area, layer 6a',
 1085: 'Secondary motor area, 

In [5]:
img = nb.load('/home/julia/data/gradients/atlas/allen_api/cortex_mask_tight.nii.gz')
hdr = img.header
aff = img.affine

In [6]:
hpc_mask, _ = mcc.get_structure_mask(1080, '/home/julia/data/gradients/atlas/allen_api/hippocampus.nrrd')
piri_mask, _ = mcc.get_structure_mask(961, '/home/julia/data/gradients/atlas/allen_api/piriform.nrrd')

In [7]:
nb.Nifti1Image(hpc_mask, aff, hdr).to_filename('/home/julia/data/gradients/atlas/allen_api/hippocampus.nii.gz')
nb.Nifti1Image(piri_mask, aff, hdr).to_filename('/home/julia/data/gradients/atlas/allen_api/piriform.nii.gz')

In [9]:
resamp = fsl.FLIRT(in_file="/home/julia/data/gradients/atlas/allen_api/hippocampus.nii.gz",
                   reference="/home/julia/data/gradients/atlas/allen_api/hippocampus.nii.gz",
                   apply_isoxfm=0.2,
                   interp="nearestneighbour",
                   out_file="/home/julia/data/gradients/atlas/allen_api/regions/hippocampus_200um.nii.gz")
resamp.run()

In [10]:
resamp = fsl.FLIRT(in_file="/home/julia/data/gradients/atlas/allen_api/piriform.nii.gz",
                   reference="/home/julia/data/gradients/atlas/allen_api/piriform.nii.gz",
                   apply_isoxfm=0.2,
                   interp="nearestneighbour",
                   out_file="/home/julia/data/gradients/atlas/allen_api/regions/piriform_200um.nii.gz")
resamp.run()

In [53]:
mask_img = nb.load('/home/julia/data/gradients/atlas/allen_api/cortex_mask_tight_200um.nii.gz')
mask_200 = mask_img.get_data()
hdr = mask_img.header
aff = mask_img.affine

In [8]:
hpc_mask_200 = nb.load("/home/julia/data/gradients/atlas/allen_api/regions/hippocampus_200um.nii.gz").get_data()
piri_mask_200 = nb.load("/home/julia/data/gradients/atlas/allen_api/regions/piriform_200um.nii.gz").get_data()

In [9]:
full_mask = np.zeros_like(mask_200)
full_mask[hpc_mask_200==1] = 1
full_mask[piri_mask_200==1] = 1
full_mask[mask_200==1] = 1

In [6]:
dist_upper = pdist(np.asarray(np.where(full_mask==1)).T,'euclidean')

In [7]:
full_corr = np.zeros((np.where(full_mask==1)[0].shape[0], np.where(full_mask==1)[0].shape[0]))
full_corr[np.triu_indices_from(full_corr, k=1)] = np.nan_to_num(dist_upper)
full_corr += full_corr.T

In [5]:
f = h5py.File('/home/julia/data/gradients/results/distance/euclidian_moieties.hdf5', 'w')
f.create_dataset('corr', data=full_corr)
f.close()

In [3]:
f = h5py.File('/home/julia/data/gradients/results/distance/euclidian_moieties.hdf5', 'r')
full_corr = np.asarray(f['corr'])
f.close()

In [41]:
full_idx = np.asarray(np.where(full_mask==1)).T
hpc_idx = np.asarray(np.where(hpc_mask_200==1)).T
piri_idx = np.asarray(np.where(piri_mask_200==1)).T

In [32]:
meta_idx = []
for i in range(hpc_idx.shape[0]):
    for j in range(full_idx.shape[0]):
        if np.all(hpc_idx[i,:] == full_idx[j,:]):
            meta_idx.append(j)
hpc_corr = full_corr[meta_idx,:]
np.save('/home/julia/data/gradients/results/distance/euclidian_hpc.npy', hpc_corr)

In [42]:
meta_idx = []
for i in range(piri_idx.shape[0]):
    for j in range(full_idx.shape[0]):
        if np.all(piri_idx[i,:] == full_idx[j,:]):
            meta_idx.append(j)
piri_corr = full_corr[meta_idx,:]
np.save('/home/julia/data/gradients/results/distance/euclidian_piri.npy', piri_corr)

In [55]:
hpc_dist = np.zeros_like(full_mask)
hpc_dist[full_mask==1]=np.min(hcp_corr, axis=0) 
hpc_dist[hpc_mask_200==1]=0
hpc_dist[piri_mask_200==1]=0
nb.Nifti1Image(hpc_dist, aff, hdr).to_filename('/home/julia/data/gradients/results/distance/euclidian_hpc.nii.gz')

In [93]:
hpc_dist = np.zeros_like(full_mask)
hpc_dist[full_mask==1]=np.min(hcp_corr, axis=0) - (np.max(np.min(hcp_corr, axis=0)) /2 )
hpc_dist[hpc_mask_200==1]=0
hpc_dist[piri_mask_200==1]=0
nb.Nifti1Image(hpc_dist, aff, hdr).to_filename('/home/julia/data/gradients/results/distance/euclidian_hpc_nonzero.nii.gz')

In [94]:
piri_dist = np.zeros_like(full_mask)
piri_dist[full_mask==1]=np.min(piri_corr, axis=0) - np.mean(np.min(piri_corr, axis=0))
piri_dist[hpc_mask_200==1]=0
piri_dist[piri_mask_200==1]=0
nb.Nifti1Image(piri_dist, aff, hdr).to_filename('/home/julia/data/gradients/results/distance/euclidian_piri_nonzero.nii.gz')

In [ ]:
piri_dist = np.zeros_like(full_mask)
piri_dist[full_mask==1]=np.min(piri_corr, axis=0) 
piri_dist[hpc_mask_200==1]=0
piri_dist[piri_mask_200==1]=0
nb.Nifti1Image(piri_dist, aff, hdr).to_filename('/home/julia/data/gradients/results/distance/euclidian_piri.nii.gz')

In [63]:
gradients = np.load('/home/julia/data/gradients/results/embedding_vol/embed.npy')

In [68]:
for g in range(6):
    r = stats.spearmanr(hpc_dist[mask_200==1], gradients[:,g])
    print('HPC', 'Gradient %i' %g, r[0])

HPC Gradient 0 -0.7176781690296353
HPC Gradient 1 0.19953978648130302
HPC Gradient 2 0.1509479889328102
HPC Gradient 3 -0.21943937385569381
HPC Gradient 4 0.24101094859748787
HPC Gradient 5 -0.11260134378388653


In [69]:
for g in range(6):
    r = stats.spearmanr(piri_dist[mask_200==1], gradients[:,g])
    print('Piri', 'Gradient %i' %g, r[0])

Piri Gradient 0 0.6646188011232524
Piri Gradient 1 -0.2178446245160824
Piri Gradient 2 0.2525939379766032
Piri Gradient 3 -0.18619311095338803
Piri Gradient 4 -0.6064007267376255
Piri Gradient 5 0.1142412771451198


In [83]:
#dist_combined = np.min(np.vstack((hpc_dist[mask_200==1], piri_dist[mask_200==1])).T, axis=1)

In [87]:
dist_combined = - hpc_dist[mask_200==1] + piri_dist[mask_200==1]

In [88]:
for g in range(6):
    r = stats.spearmanr(dist_combined, gradients[:,g])
    print('Combined', 'Gradient %i' %g, r[0])

Combined Gradient 0 0.7767389150773304
Combined Gradient 1 -0.22132475794553716
Combined Gradient 2 0.11971648615177026
Combined Gradient 3 -0.0761827668685198
Combined Gradient 4 -0.5441571792455601
Combined Gradient 5 0.09966178133337188


In [89]:
comb_dist = np.zeros_like(full_mask)
comb_dist[mask_200==1]=dist_combined
nb.Nifti1Image(comb_dist, aff, hdr).to_filename('/home/julia/data/gradients/results/distance/euclidian_comb.nii.gz')

In [90]:
comb_dist = np.zeros_like(full_mask)
comb_dist[mask_200==1]=dist_combined-np.mean(dist_combined)
nb.Nifti1Image(comb_dist, aff, hdr).to_filename('/home/julia/data/gradients/results/distance/euclidian_comb_zeromean.nii.gz')

In [101]:
mesh = io_mesh.load_mesh('/home/julia/data/gradients/atlas/allen_api/regions/annot_finest.vtk')
gradients = io_mesh.load_mesh('/home/julia/data/gradients/results/embedding_vol/embed_sampled_mesh.vtk')

In [102]:
for g in range(6):
    for a in range(3):
        r = stats.spearmanr(gradients['data'][:,g][cortex], mesh['points'][cortex][:,a])
        print('Gradient %s' %g, 'Axis %s' %a, r)

Gradient 0 Axis 0 SpearmanrResult(correlation=0.39825298612087257, pvalue=0.0)
Gradient 0 Axis 1 SpearmanrResult(correlation=-0.7618417068196369, pvalue=0.0)
Gradient 0 Axis 2 SpearmanrResult(correlation=0.00680747096807299, pvalue=0.45859419953456537)
Gradient 1 Axis 0 SpearmanrResult(correlation=-0.2667680806665898, pvalue=2.568768383733895e-192)
Gradient 1 Axis 1 SpearmanrResult(correlation=-0.016498822354997958, pvalue=0.07242910900519443)
Gradient 1 Axis 2 SpearmanrResult(correlation=-0.05718423775145105, pvalue=4.6357688329967007e-10)
Gradient 2 Axis 0 SpearmanrResult(correlation=-0.4869299111675134, pvalue=0.0)
Gradient 2 Axis 1 SpearmanrResult(correlation=-0.2381491813457714, pvalue=1.5706880605160504e-152)
Gradient 2 Axis 2 SpearmanrResult(correlation=0.047148492530618875, pvalue=2.805121048756441e-07)
Gradient 3 Axis 0 SpearmanrResult(correlation=0.11637483193617153, pvalue=4.9830389429719995e-37)
Gradient 3 Axis 1 SpearmanrResult(correlation=0.45568770717340396, pvalue=0.0)
